# **ETL Games**

<div style="text-align: justify">
En este notebook de Python, se detallarán los pasos necesarios para llevar a cabo un proceso de ETL (Extract, Transform, Load) en un dataframe que denominaremos "games". Los datos que utilizaremos provienen originalmente de un archivo JSON llamado "output_steam_games.json". A lo largo de este documento, se explorarán las etapas de extracción, transformación y carga de datos con el objetivo de preparar el conjunto de datos para su posterior análisis y uso en aplicaciones o modelos de aprendizaje automático.
</div>

## 1. Importación de las librerias

<div style="text-align: justify">
Importar bibliotecas en un ETL es esencial para aprovechar las funcionalidades predefinidas que simplifican tareas como la extracción, transformación y carga de datos. Esto ahorra tiempo y esfuerzo, asegurando un desarrollo eficiente y la creación de procesos ETL más sólidos y confiables.
</div>

In [43]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import funciones_basicas as fb

## 2. Carga de los datos

En estas lineas de código cargaremos los datos y creamos el dataframe de pandas en el que vamos a realizar el proceso de ETL, todo basado en un arhivo llamado *output_steam_juegos.json*

In [44]:
ruta_archivo =  "Datasets\output_steam_games.json"
nombre_dataframe = 'games'

games = fb.cargar_datos_json(ruta_archivo, nombre_dataframe)
games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3. Transformaciones

<div style='text-align: justify'>
En esta sección realizaremos las primeras transformaciones, estas transformaciones se realizan como una buena practica y son transversales a cualquier tipo de datos no importa su contenido, posterior a esas transformacioens inspeccionaremos a fondo la información del dataframe y realizaremos todos los procesos correspondientes, partiendo de ahi. 
</div>

In [45]:
# En esta linea, eliminaremos todos los datos que no tengan un ID.
# La razon para esto es que no podemos rellenar esos datos de ninguna manera y no nos permitiria continuar correctamente
# Eliminar filas duplicadas y nulas
games = games.dropna(subset=['id']).drop_duplicates(subset=['id']).reset_index(drop=True)

In [46]:
# Ahora verificaremos la cantidad de datos vacios que podemos encontrar en cada una de las columnas
fb.porcentaje_datos_vacios(games)

Porcentaje de datos vacíos por columna en el DataFrame:
publisher: 25.06%
genres: 10.21%
app_name: 0.00%
title: 6.38%
url: 0.00%
release_date: 6.43%
tags: 0.50%
reviews_url: 0.00%
specs: 2.08%
price: 4.29%
early_access: 0.00%
id: 0.00%
developer: 10.26%
Total de datos vacíos en el DataFrame: 5.02%


In [47]:
# Verificamos toda la información de las columnas del dataframe
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32132 entries, 0 to 32131
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24081 non-null  object
 1   genres        28850 non-null  object
 2   app_name      32131 non-null  object
 3   title         30083 non-null  object
 4   url           32132 non-null  object
 5   release_date  30066 non-null  object
 6   tags          31970 non-null  object
 7   reviews_url   32132 non-null  object
 8   specs         31463 non-null  object
 9   price         30755 non-null  object
 10  early_access  32132 non-null  object
 11  id            32132 non-null  object
 12  developer     28834 non-null  object
dtypes: object(13)
memory usage: 3.2+ MB


In [48]:
# Limpieza y manipulación de texto para la columna genres
games['genres'] = games['genres'].apply(fb.limpiar_generos)

In [49]:
# Normalizar los valores de texto en el DataFrame
for columna in games.columns:
    if games[columna].dtype == 'object':
        games[columna] = games[columna].apply(fb.normalizar_texto)

In [50]:
# Reemplazo de valores nulos en columna 'desarrollador' con valores de columna 'editor'
games['developer'] = games['developer'].fillna(games['publisher'])

# Eliminación de columnas no necesarias
games.drop(['publisher', 'early_access', 'url', 'reviews_url', 'title', 'specs', 'tags'], axis=1, inplace=True)

# Convertir listas en cadenas separadas por comas
games['genres'] = games['genres'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)

In [51]:
# Manipulación de fechas
games['release_date'] = pd.to_datetime(games['release_date'], errors='coerce')
media_fecha = games['release_date'].mean()
games['release_date'] = games['release_date'].fillna(media_fecha)
games['year'] = games['release_date'].dt.year
games.drop('release_date', axis=1, inplace=True)

In [52]:
# Conversión de tipos de datos
games['price'] = pd.to_numeric(games['price'], errors='coerce')

In [53]:
# Agrupar por las columnas "genres" y "release_date" y calcular la media de los precios
precios_por_categoria_y_fecha = games.groupby(['genres', 'year'])['price'].mean()

# Rellenar valores nulos en la columna "price"
games['price'] = games.apply(fb.rellenar_nulos, axis=1, precios_por_categoria_y_fecha=precios_por_categoria_y_fecha)

In [54]:
# Por motivos de verificación ahora veremos de nuevo la cantidad de datos vacios
fb.porcentaje_datos_vacios(games)

Porcentaje de datos vacíos por columna en el DataFrame:
genres: 0.00%
app_name: 0.00%
price: 0.06%
id: 0.00%
developer: 10.06%
year: 0.00%
Total de datos vacíos en el DataFrame: 1.69%


In [55]:
# Ahora para normalizar aun más los datos, cambiaremos el nombre de las columnas para facilitar su entendimiento
nuevos_nombres = {'genres': 'GENEROS',
                  'app_name': 'NOMBRE_JUEGO',
                  'price': 'PRECIO',
                  'id': 'ID_GAME',
                  'developer': 'DEVELOPER',
                  'year': 'AÑO'}

games = games.rename(columns=nuevos_nombres)
games.head(3)

,GENEROS,NOMBRE_JUEGO,PRECIO,ID_GAME,DEVELOPER,AÑO
0,ACTION,LOST SUMMONER KITTY,4.990000,761140,KOTOSHIRO,2018
1,FREE TO PLAY,IRONBOUND,26.790000,643980,SECRET LEVEL SRL,2018
2,CASUAL,REAL POOL 3D - POOLIANS,4.801037,670290,POOLIANS.COM,2017


## 4. Guardamos el archivo

<div style="text-align: justify">
Despues de realizadas todas las transformaciones necesarias para manejar el dataframe en futuros analisis, procedemos a guardar el documento en un formato tipo parquet para que su lectura y escritura sea más sencilla
</div>

In [56]:
fb.guardar_datos(games, "PARQUET\GAMES.parquet")